In [ ]:
import pandas as pd
import numpy as np
import duckdb
from statsmodels.stats.proportion import proportions_ztest
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_PATH = "/content/drive/MyDrive/e-Commerce Project/"
users = pd.read_csv(BASE_PATH + "users.csv", parse_dates=["signup_date"])
transactions = pd.read_csv(BASE_PATH + "transactions.csv", parse_dates=["transaction_date"])
retention = pd.read_csv(BASE_PATH + "retention.csv")

In [ ]:
users.head()

,user_id,signup_date,acquisition_channel,country,device_type,user_segment,experiment_group,activated,purchased
0,1,2024-04-12,organic,US,desktop,price_sensitive,control,True,True
1,2,2024-06-28,referral,India,mobile,power,control,False,False
2,3,2024-04-02,paid,UK,mobile,casual,treatment,False,False
3,4,2024-01-15,organic,US,desktop,price_sensitive,treatment,False,False
4,5,2024-04-16,paid,US,mobile,price_sensitive,treatment,False,False


In [ ]:
con = duckdb.connect(database=':memory:')
con.register("users", users)
con.register("transactions", transactions)
con.register("retention", retention)

**FUNNEL ANALYSIS**

In [ ]:
funnel_overall = con.execute("""
SELECT
   COUNT(*) AS signups,
   SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END) AS activated_users,
   COUNT(DISTINCT(T.user_id)) AS purchasers,
   ROUND(SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END)/COUNT(*), 4) AS activation_rate,
   ROUND(COUNT(DISTINCT(T.user_id))/SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END), 4) AS purchase_rate_from_activated,
   ROUND(COUNT(DISTINCT(T.user_id))/COUNT(*), 4) AS overall_conversion

FROM users U
LEFT JOIN transactions T ON U.user_id = T.user_id
                  """).df()
funnel_overall

,signups,activated_users,purchasers,activation_rate,purchase_rate_from_activated,overall_conversion
0,50000,18874.0,5277,0.3775,0.2796,0.1055


In [ ]:
funnel_by_device = con.execute("""
    SELECT
        device_type,
        COUNT(*) AS signups,
        SUM(CASE WHEN activated THEN 1 ELSE 0 END) AS activated_users,
        COUNT(DISTINCT t.user_id) AS purchasers,
        ROUND(
            COUNT(DISTINCT t.user_id) * 1.0 / COUNT(*),
            4
        ) AS overall_conversion
    FROM users u
    LEFT JOIN transactions t
        ON u.user_id = t.user_id
    GROUP BY device_type
    ORDER BY overall_conversion DESC;
""").df()

funnel_by_device

,device_type,signups,activated_users,purchasers,overall_conversion
0,desktop,20148,8239.0,2320,0.1151
1,mobile,29852,10635.0,2957,0.0991


In [ ]:
funnel_by_acquisition = con.execute("""
SELECT
   U.acquisition_channel,
   COUNT(*) AS signups,
   SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END) AS activated_users,
   COUNT(DISTINCT(T.user_id)) AS purchasers,
   ROUND(SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END)/COUNT(*), 4) AS activation_rate,
   ROUND(COUNT(DISTINCT(T.user_id))/SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END), 4) AS purchase_rate_from_activated,
   ROUND(COUNT(DISTINCT(T.user_id))/COUNT(*), 4) AS overall_conversion
FROM
   Users U
LEFT JOIN Transactions T ON U.user_id = T.user_id
GROUP BY
    U.acquisition_channel
""").df()
funnel_by_acquisition.head()

,acquisition_channel,signups,activated_users,purchasers,activation_rate,purchase_rate_from_activated,overall_conversion
0,organic,20029,7490.0,2088,0.3740,0.2788,0.1042
1,paid,20059,7556.0,2081,0.3767,0.2754,0.1037
2,referral,9912,3828.0,1108,0.3862,0.2894,0.1118


In [ ]:
funnel_by_user_segment = con.execute("""
SELECT
   U.user_segment,
   COUNT(*) AS signups,
   SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END) AS activated_users,
   COUNT(DISTINCT(T.user_id)) AS purchases,
   ROUND(SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END)/COUNT(*), 4) AS activation_rate,
   ROUND(COUNT(DISTINCT(T.user_id))/SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END), 4) AS purchase_rate_from_activated,
   ROUND(COUNT(DISTINCT(T.user_id))/COUNT(*), 4) AS overall_conversion
FROM
   Users U
LEFT JOIN Transactions T ON U.user_id = T.user_id
GROUP BY
   U.user_segment
""").df()
funnel_by_user_segment.head()

,user_segment,signups,activated_users,purchases,activation_rate,purchase_rate_from_activated,overall_conversion
0,price_sensitive,19954,7009.0,1917,0.3513,0.2735,0.0961
1,power,14979,6708.0,1936,0.4478,0.2886,0.1292
2,casual,15067,5157.0,1424,0.3423,0.2761,0.0945


In [ ]:
funnel_by_country = con.execute("""
SELECT
   U.country,
   COUNT(*) As signups,
   SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END) AS activated_users,
   COUNT(DISTINCT(T.user_id)) AS purchases,
   ROUND(SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END)/COUNT(*), 4) AS activation_rate,
   ROUND(COUNT(DISTINCT(T.user_id))/SUM(CASE WHEN U.activated = 'True' THEN 1 ELSE 0 END), 4) AS purchase_rate_from_activated,
   ROUND(COUNT(DISTINCT(T.user_id))/COUNT(*), 4) AS overall_conversion
FROM
   Users U
LEFT JOIN
   Transactions T ON U.user_id = T.user_id
GROUP BY
   U.country
""").df()

funnel_by_country

,country,signups,activated_users,purchases,activation_rate,purchase_rate_from_activated,overall_conversion
0,India,15017,5637.0,1577,0.3754,0.2798,0.1050
1,US,24988,9469.0,2651,0.3789,0.2800,0.1061
2,UK,9995,3768.0,1049,0.3770,0.2784,0.1050


**AB TESTING**

In [ ]:
ab_summary = con.execute("""
    SELECT
        experiment_group,
        COUNT(*) AS users,
        SUM(CASE WHEN activated THEN 1 ELSE 0 END) AS activated_users,
        COUNT(DISTINCT t.user_id) AS purchasers,

        ROUND(
            SUM(CASE WHEN activated THEN 1 ELSE 0 END) * 1.0 / COUNT(*),
            4
        ) AS activation_rate,

        ROUND(
            COUNT(DISTINCT t.user_id) * 1.0 / COUNT(*),
            4
        ) AS overall_conversion

    FROM users u
    LEFT JOIN transactions t
        ON u.user_id = t.user_id
    GROUP BY experiment_group;
""").df()

ab_summary

,experiment_group,users,activated_users,purchasers,activation_rate,overall_conversion
0,control,24908,8807.0,2501,0.3536,0.1004
1,treatment,25092,10067.0,2776,0.4012,0.1106


In [ ]:
control = users[users["experiment_group"] == "control"]
treatment = users[users["experiment_group"] == "treatment"]

successes = np.array([
    control["activated"].sum(),
    treatment["activated"].sum()
])

trials = np.array([
    len(control),
    len(treatment)
])

stat, pval = proportions_ztest(successes, trials)

stat, pval

(np.float64(-10.983471940771768), np.float64(4.5893918184148145e-28))

In [ ]:
control_rate = control["activated"].mean()
treatment_rate = treatment["activated"].mean()

lift = treatment_rate - control_rate
relative_lift = lift / control_rate

control_rate, treatment_rate, lift, relative_lift

(np.float64(0.35358117873775496),
 np.float64(0.401203570859238),
 np.float64(0.04762239212148306),
 np.float64(0.13468587975041443))

In [ ]:
segment_effect = con.execute("""
    SELECT
        device_type,
        experiment_group,
        COUNT(*) AS users,
        ROUND(
            SUM(CASE WHEN activated THEN 1 ELSE 0 END) * 1.0 / COUNT(*),
            4
        ) AS activation_rate
    FROM users
    GROUP BY device_type, experiment_group
    ORDER BY device_type, experiment_group;
""").df()

segment_effect

,device_type,experiment_group,users,activation_rate
0,desktop,control,10048,0.3826
1,desktop,treatment,10100,0.4351
2,mobile,control,14860,0.3340
3,mobile,treatment,14992,0.3783


**COHORT AND RETENTION ANALYSIS**

In [ ]:
retention_enriched = retention.merge(
    users[
        ["user_id", "signup_date", "experiment_group",
         "acquisition_channel", "device_type", "user_segment"]
    ],
    on="user_id",
    how="left"
)

retention_enriched["signup_month"] = (
    retention_enriched["signup_date"]
    .dt.to_period("M")
    .astype(str)
)

overall_retention = (
    retention_enriched
    .groupby("week")["is_active"]
    .mean()
    .reset_index()
    .rename(columns={"is_active": "retention_rate"})
)

overall_retention

,week,retention_rate
0,0,0.92944
1,1,0.86710
2,2,0.81000
3,3,0.75374
4,4,0.70374
5,5,0.65624
6,6,0.61192
7,7,0.57176
8,8,0.53590
9,9,0.50328


In [ ]:
retention_by_experiment = (
    retention_enriched
    .groupby(["experiment_group", "week"])["is_active"]
    .mean()
    .reset_index()
    .rename(columns={"is_active": "retention_rate"})
)

retention_by_experiment

,experiment_group,week,retention_rate
0,control,0,0.937972
1,control,1,0.883331
2,control,2,0.832463
3,control,3,0.783403
4,control,4,0.738598
5,control,5,0.697005
6,control,6,0.658303
7,control,7,0.620724
8,control,8,0.586960
9,control,9,0.556689


In [ ]:
cohort_table = (
    retention_enriched
    .groupby(["signup_month", "week"])["is_active"]
    .mean()
    .reset_index()
)

cohort_table

,signup_month,week,is_active
0,2024-01,0,0.930108
1,2024-01,1,0.868356
2,2024-01,2,0.812442
3,2024-01,3,0.752842
4,2024-01,4,0.705991
...,...,...,...
91,2024-08,7,0.579769
92,2024-08,8,0.543212
93,2024-08,9,0.512866
94,2024-08,10,0.481988


In [ ]:
segment_retention = (
    retention_enriched
    .groupby(["user_segment", "week"])["is_active"]
    .mean()
    .reset_index()
)

segment_retention

,user_segment,week,is_active
0,casual,0,0.948098
1,casual,1,0.901042
2,casual,2,0.860158
3,casual,3,0.816155
4,casual,4,0.776001
5,casual,5,0.733988
6,casual,6,0.695493
7,casual,7,0.662242
8,casual,8,0.631645
9,casual,9,0.605164


In [ ]:
early_churn = (
    retention_enriched
    .query("week <= 3")
    .groupby("experiment_group")["is_active"]
    .mean()
    .reset_index()
)

early_churn

,experiment_group,is_active
0,control,0.859292
1,treatment,0.820989


**LTV ANALYSIS**

In [ ]:
user_revenue = (
    transactions
    .groupby("user_id")["revenue"]
    .sum()
    .reset_index()
    .rename(columns={"revenue": "total_revenue"})
)

users_revenue = users.merge(
    user_revenue,
    on="user_id",
    how="left"
)

users_revenue["total_revenue"] = users_revenue["total_revenue"].fillna(0)   # After left join, some users may not be present in the users table. For them fill null values with 0.
users_revenue

,user_id,signup_date,acquisition_channel,country,device_type,user_segment,experiment_group,activated,purchased,total_revenue
0,1,2024-04-12,organic,US,desktop,price_sensitive,control,True,True,20.0
1,2,2024-06-28,referral,India,mobile,power,control,False,False,0.0
2,3,2024-04-02,paid,UK,mobile,casual,treatment,False,False,0.0
3,4,2024-01-15,organic,US,desktop,price_sensitive,treatment,False,False,0.0
4,5,2024-04-16,paid,US,mobile,price_sensitive,treatment,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...
49995,49996,2024-03-22,paid,US,desktop,price_sensitive,control,False,False,0.0
49996,49997,2024-03-15,organic,India,desktop,casual,control,True,False,0.0
49997,49998,2024-06-21,organic,India,mobile,price_sensitive,treatment,True,True,20.0
49998,49999,2024-07-22,paid,US,desktop,casual,treatment,False,False,0.0


In [ ]:
total_revenue = users_revenue["total_revenue"].sum()
arpu = users_revenue["total_revenue"].mean()
paying_arpu = users_revenue.query("total_revenue > 0")["total_revenue"].mean()

total_revenue, arpu, paying_arpu

(np.float64(392280.0), np.float64(7.8456), np.float64(74.3376918703809))

In [ ]:
revenue_by_experiment = (
    users_revenue
    .groupby("experiment_group")
    .agg(
        users=("user_id", "count"),
        total_revenue=("total_revenue", "sum"),
        arpu=("total_revenue", "mean"),
        paying_users=("total_revenue", lambda x: (x > 0).sum())
    )
    .reset_index()
)

revenue_by_experiment

,experiment_group,users,total_revenue,arpu,paying_users
0,control,24908,183580.0,7.370323,2501
1,treatment,25092,208700.0,8.317392,2776


In [ ]:
users_revenue["signup_month"] = (
    users_revenue["signup_date"]
    .dt.to_period("M")
    .astype(str)
)

cohort_revenue = (
    users_revenue
    .groupby("signup_month")
    .agg(
        users=("user_id", "count"),
        total_revenue=("total_revenue", "sum"),
        arpu=("total_revenue", "mean")
    )
    .reset_index()
    .sort_values("signup_month")
)

cohort_revenue

,signup_month,users,total_revenue,arpu
0,2024-01,6510,48340.0,7.425499
1,2024-02,5977,47260.0,7.906977
2,2024-03,6521,48660.0,7.462046
3,2024-04,6323,51200.0,8.097422
4,2024-05,6500,52680.0,8.104615
5,2024-06,6227,49000.0,7.868958
6,2024-07,6307,50820.0,8.057714
7,2024-08,5635,44320.0,7.865129


In [ ]:
segment_revenue = (
    users_revenue
    .groupby("user_segment")
    .agg(
        users=("user_id", "count"),
        total_revenue=("total_revenue", "sum"),
        arpu=("total_revenue", "mean")
    )
    .reset_index()
    .sort_values("arpu", ascending=False)
)

segment_revenue

,user_segment,users,total_revenue,arpu
1,power,14979,145640.0,9.722945
0,casual,15067,106600.0,7.075065
2,price_sensitive,19954,140040.0,7.018142


In [ ]:
activation_revenue = (
    users_revenue
    .groupby("experiment_group")
    .agg(
        activation_rate=("activated", "mean"),
        arpu=("total_revenue", "mean")
    )
    .reset_index()
)

activation_revenue

,experiment_group,activation_rate,arpu
0,control,0.353581,7.370323
1,treatment,0.401204,8.317392


**CHURN MODELING**

In [ ]:
final_week = 8

churn_label = (
    retention
    .query(f"week == {final_week}")
    .rename(columns={"is_active": "is_active_final"})
    [["user_id", "is_active_final"]]
)

churn_label["churned"] = 1 - churn_label["is_active_final"]

In [ ]:
early_activity = (
    retention
    .query("week <= 4")
    .groupby("user_id")["is_active"]
    .mean()
    .reset_index()
    .rename(columns={"is_active": "early_activity_rate"})
)
early_activity

,user_id,early_activity_rate
0,1,1.0
1,2,0.0
2,3,0.0
3,4,1.0
4,5,1.0
...,...,...
49995,49996,1.0
49996,49997,1.0
49997,49998,0.6
49998,49999,1.0


In [ ]:
features = (
    users
    .merge(early_activity, on="user_id", how="left")
    .merge(churn_label[["user_id", "churned"]], on="user_id", how="left")
)

features["early_activity_rate"] = features["early_activity_rate"].fillna(0)

In [ ]:
model_df = features[[
    "churned",
    "activated",
    "early_activity_rate",
    "experiment_group",
    "device_type",
    "user_segment"
]]

In [ ]:
model_df = pd.get_dummies(
    model_df,
    columns=["experiment_group", "device_type", "user_segment"],
    drop_first=True
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = model_df.drop("churned", axis=1)
y = model_df["churned"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_prob = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)

auc

np.float64(0.8640700493655846)

In [ ]:
coefficients = (
    pd.DataFrame({
        "feature": X.columns,
        "coefficient": model.coef_[0]
    })
    .sort_values("coefficient", ascending=False)
)

coefficients

,feature,coefficient
5,user_segment_price_sensitive,0.811064
2,experiment_group_treatment,0.369989
4,user_segment_power,0.076821
3,device_type_mobile,0.025306
0,activated,0.015628
1,early_activity_rate,-19.886841
